In [1]:
#include <cmath>
#include <unordered_set>
#include <xtensor/xarray.hpp>
#include <iostream>

double r(const int& i, 
         const std::unordered_set<int>& G, 
         const xt::xarray<double>& dist) 
{    
    double max = 0;
    
    for(int j : G){
        double dij;
        if (i==j)
            dij = 0;
        else if(i < j)
            dij = dist[dist.size()*i+j-((i+2)*(i+1))/2];
        else
            dij = dist[dist.size()*j+i-((j+2)*(j+1))/2];
        
        max =  dij > max ? dij : max;
    }
    return max;
}


In [ ]:
#include <xtensor/xarray.hpp>
#include <xtensor/xnpy.hpp>
#include <unordered_set>
#include <unordered_map>
#include <algorithm>
#include <cmath>
#include <limits>
#include <iterator>

using ClusterMap = std::unordered_map<int, std::unordered_set<int>>;

// auto dist = xt::load_npy<double>("tangent_distances.npy");

auto dist = xt::load_npy<double>("toy.npy");
int n = std::round((std::sqrt(8*dist.size() + 1) + 1)/2);

xt::xarray<double> Z = xt::xarray<double>::from_shape({static_cast<unsigned long>(n-1), 4});
ClusterMap clusters;

for (int i=0 ; i<5 ; i++)
    clusters[i] = std::unordered_set<int>({i});

for (size_t i=0 ; i<n-1 ; i++)
{
    double min_d = std::numeric_limits<double>::infinity();
    xt::xarray<double> to_merge;
    
    std::cout << i << std::endl;
    
    for(ClusterMap::iterator it=clusters.begin(); 
        it!=clusters.end() ; it++){
        
        for(ClusterMap::iterator jt=clusters.begin(); 
            jt!=it ; jt++){
            
            const int& idxG = it->first;
            const int& idxH = jt->first;
            
            std::unordered_set<int>& GH = it->second;
            std::unordered_set<int>& H = jt->second;
            
            GH.merge(H);
            
            double d = std::numeric_limits<double>::infinity();
            for (const int& x : GH){
                double minimax = r(x, GH, dist);
                d = minimax < d ? minimax : d;
            }
            std::cout << i << std::endl;
            if (d < min_d){
                min_d = d;
                to_merge = {(double)idxG, (double)idxH, d, (double)GH.size()};
            }       
        }
    }     
    
    
    for (int k=0 ; k<4 ; k++)
        Z(i, k) = to_merge[k];
    
    double idxG = to_merge[0];
    double idxH = to_merge[1];
    
    std::unordered_set<int> new_cluster = clusters[idxG];
    new_cluster.merge(clusters[(int)idxH]);
    
    clusters[n+i] = new_cluster;
    clusters.erase(idxG);
    clusters.erase(idxH);
}

xt::dump_npy("Z_tan_minimax.npy", Z);

input_line_10:59:11: error: invalid operands to binary expression ('std::ostream' (aka 'basic_ostream<char>') and 'xt::xarray<double>' (aka 'xt::xarray_container<xt::uvector<double, std::allocator<double> >, xt::layout_type::row_major, xt::svector<unsigned long, 4, std::allocator<unsigned long>, true>, xt::xtensor_expression_tag>'))
std::cout << Z;
~~~~~~~~~ ^  ~
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/ostream:245:7: note: candidate function not viable: no known conversion from 'xt::xarray<double>' (aka 'xt::xarray_container<xt::uvector<double, std::allocator<double> >, xt::layout_type::row_major, xt::svector<unsigned long, 4, std::allocator<unsigned long>, true>, xt::xtensor_expression_tag>') to 'const void *' for 1st argument; take the address of the argument with &
      operator<<(const void* __p)
      ^
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/cstddef:124:5: note: candidate function not viable: no known conversion from 'std::ostr